In [29]:
import pandas as pd

# Veri setini yükleme
df = pd.read_csv("sql.csv")

# İlk birkaç satırı kontrol et
print(df.head())

# Veri boyutunu kontrol et
print(f"Veri boyutu: {df.shape}")



                                               Query  Label
0                  " or pg_sleep  (  __TIME__  )  --      1
1  create user name identified by pass123 tempora...      1
2   AND 1  =  utl_inaddr.get_host_address   (    ...      1
3   select * from users where id  =  '1' or @ @1 ...      1
4   select * from users where id  =  1 or 1#"  ( ...      1
Veri boyutu: (22913, 2)


In [30]:
# Eksik değerleri kontrol et ve temizle
print("Eksik değerler:", df.isnull().sum())
df = df.dropna()  # NaN içeren satırları kaldır

# Boş stringler veya gereksiz veriler varsa kaldır
df = df[df["Query"].str.strip() != ""]  # Boş stringleri temizle

# Label sütununun yalnızca 0 ve 1 içerdiğini doğrula
print("Label dağılımı:", df["Label"].value_counts())


Eksik değerler: Query    0
Label    0
dtype: int64
Label dağılımı: Label
0    11584
1    11329
Name: count, dtype: int64


In [31]:
from sklearn.model_selection import train_test_split

# Girdi (Query) ve hedef (Label) sütunlarını ayır
X = df["Query"]  # SQL sorguları
y = df["Label"]  # Etiketler (1: saldırgan, 0: güvenli)

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

print(f"Eğitim seti boyutu: {X_train.shape[0]}")
print(f"Test seti boyutu: {X_test.shape[0]}")


Eğitim seti boyutu: 18330
Test seti boyutu: 4583


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vektörizeri tanımlayın
vectorizer = TfidfVectorizer(max_features=100, ngram_range=(1, 2))
# max_features 5000 1000, ngram_range (1,1) (2,2)

# Eğitim ve test setini TF-IDF ile dönüştür
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"TF-IDF Eğitim seti boyutu: {X_train_tfidf.shape}")
print(f"TF-IDF Test seti boyutu: {X_test_tfidf.shape}")


TF-IDF Eğitim seti boyutu: (18330, 100)
TF-IDF Test seti boyutu: (4583, 100)


In [33]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest modelini tanımlayın
model = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=42, min_samples_split=5, min_samples_leaf=2,class_weight="balanced")

# Modeli eğitim seti ile eğitin
model.fit(X_train_tfidf, y_train)

print("Model eğitimi tamamlandı.")

Model eğitimi tamamlandı.


In [34]:
from sklearn.metrics import accuracy_score

# Test setinde tahmin yap
y_pred = model.predict(X_test_tfidf)

# Doğruluk skorunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk: {accuracy:.2f}")


Doğruluk: 0.93


In [35]:
from sklearn.metrics import classification_report

print("\nPerformans Raporu:")
print(classification_report(y_test, y_pred, target_names=["Benign", "Malicious"]))


Performans Raporu:
              precision    recall  f1-score   support

      Benign       0.91      0.96      0.94      2335
   Malicious       0.96      0.90      0.93      2248

    accuracy                           0.93      4583
   macro avg       0.94      0.93      0.93      4583
weighted avg       0.94      0.93      0.93      4583



In [36]:
import joblib, os

model_dir = "models"

joblib.dump(vectorizer, os.path.join(model_dir, f'{"sql"}_vectorizer.pkl'))
joblib.dump(model, os.path.join(model_dir, f'{"sql"}_model.pkl'))

print("Model ve TF-IDF vektörizer kaydedildi.")


Model ve TF-IDF vektörizer kaydedildi.


In [38]:
# Yeni bir SQL sorgusu
new_query = [" AND 1  =  utl_inaddr.get_host_address   (    (   SELECT DISTINCT  (  granted_role  )   FROM   (  SELECT DISTINCT  (  granted_role  )  , ROWNUM AS LIMIT FROM dba_role_privs WHERE GRANTEE  =  SYS.LOGINUSER  )   WHERE LIMIT  =  2   )    )    AND 'i'  =  'i"]

# TF-IDF ile sorguyu dönüştür
new_query_tfidf = vectorizer.transform(new_query)

# Tahmin yap
new_prediction = model.predict(new_query_tfidf)
print("Tahmin edilen sınıf:", "Malicious" if new_prediction[0] == 1 else "Benign")


Tahmin edilen sınıf: Benign


In [62]:
new_queries = [
    # Zafiyetli (SQL Injection'a açık) komutlar:
    "select * from users where id  =  1 or ""]{"" or 1  =  1 -- 1",
    "select * from users where id  =  '1' or @ @1  =  1 union select 1,version  (    )   -- 1'",
    "1"" where 9241  =  9241 union all select null#", 
    "1  )   where 3738  =  3738 or 8421  =    (  select count  (  *  )   from generate_series  (  1,5000000   )    )   --",
    # Güvenli (SQL Injection'a karşı korumalı) komutlar:
    "cursor.execute('SELECT name, age FROM employees WHERE department_id = ? AND status = ?', (department_id, 'active'))",  # Parametrik sorgu ile güvenli veri alımı
    "SELECT orders.id, customers.name FROM orders INNER JOIN customers ON orders.customer_id = customers.id WHERE customers.country = 'USA';",  # INNER JOIN ile güvenli sorgu
    "GRANT SELECT ON table_name TO 'username';",  # Veritabanı yetkisi ile güvenli kullanım
    "SELECT * FROM die FETCH FIRST 50 PERCENT ROWS ONLY",
]

# TF-IDF ile dönüştürme
new_queries_tfidf = vectorizer.transform(new_queries)

# Tahmin yapma
predictions = model.predict(new_queries_tfidf)

# Sonuçları yazdırma
for query, prediction in zip(new_queries, predictions):
    result = "Malicious" if prediction == 1 else "Benign"
    print(f"Sorgu: {query}\nTahmin edilen sınıf: {result}\n")


Sorgu: select * from users where id  =  1 or ]{ or 1  =  1 -- 1
Tahmin edilen sınıf: Malicious

Sorgu: select * from users where id  =  '1' or @ @1  =  1 union select 1,version  (    )   -- 1'
Tahmin edilen sınıf: Malicious

Sorgu: 1 where 9241  =  9241 union all select null#
Tahmin edilen sınıf: Malicious

Sorgu: 1  )   where 3738  =  3738 or 8421  =    (  select count  (  *  )   from generate_series  (  1,5000000   )    )   --
Tahmin edilen sınıf: Malicious

Sorgu: cursor.execute('SELECT name, age FROM employees WHERE department_id = ? AND status = ?', (department_id, 'active'))
Tahmin edilen sınıf: Benign

Sorgu: SELECT orders.id, customers.name FROM orders INNER JOIN customers ON orders.customer_id = customers.id WHERE customers.country = 'USA';
Tahmin edilen sınıf: Benign

Sorgu: GRANT SELECT ON table_name TO 'username';
Tahmin edilen sınıf: Benign

Sorgu: SELECT * FROM die FETCH FIRST 50 PERCENT ROWS ONLY
Tahmin edilen sınıf: Benign

